In [1]:
# pip3 install mariadb
# pip3 install pandas
# pip3 install numpy  
# sudo apt-get install libatlas-base-dev
# pip3 install matplotlib
# pip3 install ipympl
# pip3 install ipywidgets

import os
import sys
import mariadb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, IntSlider, IntRangeSlider, HBox
import ipywidgets as widgets

In [2]:
username = os.environ.get("db_user")
password = os.environ.get("db_pwd")

w_db = widgets.RadioButtons(
        options=['reformer', 'catalyst'],
        description='Select Database:',
        disabled=False
        )
display(w_db)
db_name = w_db.value

RadioButtons(description='Select Database:', options=('reformer', 'catalyst'), value='reformer')

In [3]:
try:
    # Connect to MariaDB Platform
    conn = mariadb.connect(
        user=username,
        password=password,
        host="localhost",
        port=3306,
        database=db_name,
        autocommit=True
    )

    # Get Cursor for tx
    cur = conn.cursor(named_tuple=False)
    #print(conn)
    print(cur)
    print(f"Connected to {db_name}@localhost ")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

Connected to reformer@localhost 


In [6]:
%matplotlib widget

output = widgets.Output()
with output:
    fig_TC, ax_TC = plt.subplots(constrained_layout=True, figsize=(10, 7))
fig_TC.canvas.toolbar_position = 'left'

cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Database')
cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200, description='Time Range',continuous_update=False)

def update_time_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
table_dropdown.observe(update_time_range, 'value')

def plot_TC(Table_name, Time):    
    cur.execute(f"SELECT * FROM {Table_name}")
    df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
    plt.cla()
    #ax2 = ax1.twinx()
    df['TC7'].plot(legend=True, ax=ax_TC)
    df['TC8'].plot(legend=True, ax=ax_TC)
    df['TC9'].plot(legend=True, ax=ax_TC)
    df['TC10'].plot(legend=True, ax=ax_TC)
    df['EVA_out'].plot(legend=True, ax=ax_TC,
                       title=f'TC_{Table_name}', 
                       xlabel='Time[s]', 
                       ylabel='Temp[oC]', 
                       grid=True,  
                       secondary_y=False, 
                       colormap='winter', 
                       xlim=Time, 
                       xticks=range(Time[0],Time[1],1200),
                       ylim=(0,1000), 
                       yticks=range(0,1000,50),
                      )
    #ax1.set_ylabel('Temp_SR [oC]', color='g')
    #ax2.set_ylabel('Temp_BR [oC]', color='b')
    #fig_TC.canvas.draw()
    #plt.legend(loc='upper right')
    plt.show()
    

plot = interactive_output(plot_TC, {'Table_name':table_dropdown, 'Time':time_slider})
control = HBox([table_dropdown, time_slider])
widgets.VBox([control, plot])

In [5]:
%matplotlib widget

output = widgets.Output()
with output:
    fig_DFM, ax_DFM = plt.subplots(constrained_layout=True, figsize=(10, 5))
fig_DFM.canvas.toolbar_position = 'left'

cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Database')
cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200, description='Time Range',continuous_update=False)

def update_time_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
table_dropdown.observe(update_time_range, 'value')

def plot_TC(Table_name, Time):    
    cur.execute(f"SELECT * FROM {Table_name}")
    df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
    plt.cla()
    #ax2 = ax1.twinx()
    df['DFM_RichGas'].plot(legend=True, ax=ax_DFM)
    df['DFM_AOG'].plot(legend=True, ax=ax_DFM)
    df['DFM_total'] = df['DFM_RichGas'] + df['DFM_AOG']
    df['DFM_total'].plot(legend=True, ax=ax_DFM,
                       title=f'TC_{Table_name}',
                       xlabel='Time[s]', 
                       ylabel='Temp[oC]', 
                       grid=True,  
                       secondary_y=False, 
                       colormap='winter', 
                       xlim=Time, 
                       xticks=range(Time[0],Time[1],1200),
                       ylim=(0,150), 
                       yticks=range(0,150,10),
                      )
    #ax1.set_ylabel('Temp_SR [oC]', color='g')
    #ax2.set_ylabel('Temp_BR [oC]', color='b')
    #fig_DFM.canvas.draw()
    #plt.legend(loc='upper right')
    plt.show()
    

plot = interactive_output(plot_TC, {'Table_name':table_dropdown, 'Time':time_slider})
control = HBox([table_dropdown, time_slider])
widgets.VBox([control, plot])

In [ ]:
conn.close()

In [ ]:
# create some control elements
cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Database')

cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200, description='Time Range',continuous_update=False)

color_picker = widgets.ColorPicker(value='#FF00DD', description='pick a color')

output = widgets.Output()
 
# create data
cur.execute(f"SELECT * FROM {table_dropdown.value}")
df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
x = df.index
 
with output:
    fig, ax = plt.subplots(constrained_layout=True, figsize=(10, 5))
     
# move the toolbar to the bottom
fig.canvas.toolbar_position = 'left'
ax.grid(True)    
line_TC7, = ax.plot(x, df['TC7'])
line_TC8, = ax.plot(x, df['TC8'])
line_TC9, = ax.plot(x, df['TC9'])
line_TC10, = ax.plot(x, df['TC10'])
line_EVA_out, = ax.plot(x, df['EVA_out'])


# callback functions
def update_time_range(change):
    cur.execute(f"SELECT * FROM {change.new}")
    df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
    time_slider.max = cur.rowcount
    """redraw line (update plot)"""
    x = df.index
    line_TC7, = ax.plot(x, df['TC7'])
    line_TC8, = ax.plot(x, df['TC8'])
    line_TC9, = ax.plot(x, df['TC9'])
    line_TC10, = ax.plot(x, df['TC10'])
    line_EVA_out, = ax.plot(x, df['EVA_out'])
    #fig.canvas.draw()
    
def line_color(change):
    """set line color"""
    line.set_color(change.new)

# connect callbacks and traits
table_dropdown.observe(update_time_range, 'value')
int_slider.observe(update, 'value')
color_picker.observe(line_color, 'value')

controls = widgets.HBox([table_dropdown, time_slider, color_picker])
widgets.HBox([controls, output])